In [1]:
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import train_test_split
import time
import optuna
import xgboost as xgb
from sklearn.metrics import f1_score, classification_report, confusion_matrix


In [2]:
def clean_ds (df):
    df = df.copy()
    for i in range(1, 4):  
        df[f'X_t-{i}'] = df['Close'].shift(i)

    # Shift Close Column up by 5 rows
    df['Pt_5'] = df['Close'].shift(-5)

    #Agregamos RSI
    rsi_data = ta.momentum.RSIIndicator(close= df['Close'], window=28)
    df['RSI'] = rsi_data.rsi()

    # La Y
    df['Y_BUY'] = df['Close'] < df['Pt_5']
    df['Y_SELL'] = df['Close'] > df['Pt_5']
    
    # df['Y_BUY'] = df['Y_BUY'].astype(int)
    # df['Y_SELL'] = df['Y_SELL'].astype(int)

    return df

In [90]:
data_1 = pd.read_csv('../data/aapl_5m_train.csv')
data_1

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0
...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0


In [91]:
data_ejem = clean_ds(data_1)
data_ejem

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


In [92]:
close_data = data_ejem[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']]
close_data


,Close,X_t-1,X_t-2,X_t-3,RSI,Y_BUY
0,132.809997,NaN,NaN,NaN,NaN,False
1,131.889999,132.809997,NaN,NaN,NaN,False
2,132.059997,131.889999,132.809997,NaN,NaN,False
3,132.250000,132.059997,131.889999,132.809997,NaN,False
4,131.589996,132.250000,132.059997,131.889999,NaN,False
...,...,...,...,...,...,...
39574,129.054992,129.039993,129.020004,128.910003,56.509324,False
39575,129.380004,129.054992,129.039993,129.020004,60.700323,False
39576,129.764404,129.380004,129.054992,129.039993,64.854367,False
39577,129.949996,129.764404,129.380004,129.054992,66.620919,False


In [93]:
close_data['Y_BUY'].value_counts()


Y_BUY
True     20037
False    19542
Name: count, dtype: int64

In [94]:
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]

In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [96]:
class XGBOptimizer:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def opt_xgb(self, trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 5, 150),
            'max_depth': trial.suggest_int('max_depth', 3, 100),
            'max_leaves': trial.suggest_int('max_leaves', 3, 100),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 2),
            'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
            'gamma': trial.suggest_float('gamma', 0.01, 50),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10),
            'random_state': 42,
        }
        model = xgb.XGBClassifier(**params)
        model.fit(self.x_train, self.y_train)
        y_pred = model.predict(self.x_test)
        f1 = f1_score(self.y_test, y_pred)
        return f1

    def xgb_optuna(self):
        start_time = time.time()
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: self.opt_xgb(trial), n_trials=50)
        trial = study.best_trial
        print('F1: {}'.format(trial.value))
        print("Best hyperparameters: {}".format(trial.params))
        end_time = time.time()
        execution_time_minutes = (end_time - start_time) / 60
        print("Execution time: {} minutes".format(execution_time_minutes))
        return trial.params


In [97]:
XGBOptimizer(x_train, y_train, x_test, y_test).xgb_optuna()

[I 2024-03-07 18:42:19,419] A new study created in memory with name: no-name-8b500633-27d5-4756-aabd-bd66864df0ef
[I 2024-03-07 18:42:19,845] Trial 0 finished with value: 0.5361642557162856 and parameters: {'n_estimators': 106, 'max_depth': 70, 'max_leaves': 94, 'learning_rate': 1.360567556500264, 'booster': 'gbtree', 'gamma': 5.5592322441483235, 'reg_alpha': 4.255152715214858, 'reg_lambda': 7.169300373476912}. Best is trial 0 with value: 0.5361642557162856.
[I 2024-03-07 18:42:21,514] Trial 1 finished with value: 0.6719234963509773 and parameters: {'n_estimators': 37, 'max_depth': 65, 'max_leaves': 13, 'learning_rate': 1.626617018619086, 'booster': 'dart', 'gamma': 31.95434072636725, 'reg_alpha': 0.558143181558517, 'reg_lambda': 7.57463627214361}. Best is trial 1 with value: 0.6719234963509773.
[I 2024-03-07 18:42:21,642] Trial 2 finished with value: 0.6719234963509773 and parameters: {'n_estimators': 75, 'max_depth': 55, 'max_leaves': 6, 'learning_rate': 1.0159977067940316, 'booster'

F1: 0.6719234963509773
Best hyperparameters: {'n_estimators': 37, 'max_depth': 65, 'max_leaves': 13, 'learning_rate': 1.626617018619086, 'booster': 'dart', 'gamma': 31.95434072636725, 'reg_alpha': 0.558143181558517, 'reg_lambda': 7.57463627214361}
Execution time: 2.254512544473012 minutes


{'n_estimators': 37,
 'max_depth': 65,
 'max_leaves': 13,
 'learning_rate': 1.626617018619086,
 'booster': 'dart',
 'gamma': 31.95434072636725,
 'reg_alpha': 0.558143181558517,
 'reg_lambda': 7.57463627214361}

In [109]:
import xgboost as xgb

# Define the hyperparameters
hyperparameters = {'n_estimators': 37,
 'max_depth': 65,
 'max_leaves': 13,
 'learning_rate': 1.626617018619086,
 'booster': 'dart',
 'gamma': 31.95434072636725,
 'reg_alpha': 0.558143181558517,
 'reg_lambda': 7.57463627214361}

# Create the XGBoost model
model = xgb.XGBClassifier(random_state=42, **hyperparameters)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

       False       0.00      0.00      0.00      3911
        True       0.51      1.00      0.67      4005

    accuracy                           0.51      7916
   macro avg       0.25      0.50      0.34      7916
weighted avg       0.26      0.51      0.34      7916



/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rodo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [110]:
confusion_matrix(y_test, y_pred)

array([[   0, 3911],
       [   0, 4005]])

In [111]:
y_test.sum() / len(y_test)

Y_BUY    0.505937
dtype: float64

In [112]:
y_pred.sum() / len(y_pred)

1.0